In [ ]:
import transformers
import torch

model_id = "MLP-KTLim/llama-3-Korean-Bllossom-8B"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
    max_length=1000
)

In [ ]:
terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

In [ ]:
novel_prompt = """너는 단문 소설을 쓰는 작가야. 주어진 단어를 포함하는 세 문단짜리 단문 소설을 작문해야 해.
아래 단어를 모두 포함하도록 단문 소설을 써 줘.
{keywords}
제목 없이 소설 내용만 써 줘. 주어진 단어를 활용하는 자연스러운 전개로 써 줘.
전지적 작가 시점으로 써 줘.
인물의 대사 한 개 이상을 포함해.
소설은 세 문단으로 이루어져야 해. 세 문단 이상 쓰지 마.
"""
science_prompt = """너는 과학적 사실을 설명하는 글을 쓰는 과학 저널리스트야. 주어진 과학 질문에 대한 답변을 세 문단으로 작성해야 해.
아래 과학 질문에 대한 답변을 써 줘.
{question}
제목이나 요약 없이 답변 내용만 써 줘. 질문과 관련없는 내용은 쓰지 마.
-다, 와 같은 종결어미를 사용하는 해라체로 작성해.
답변은 세 문단으로 이루어져야 해.
"""

In [ ]:
import csv

results = []
with open('hw4_data/sample_novel.csv', 'r', newline='') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        instruction = novel_prompt.format(keywords=row)
        messages = [
            {"role": "system", "content": "당신은 유능한 AI 어시스턴트 입니다. 사용자의 질문에 대해 친절하게 답변해주세요."},
            {"role": "user", "content": f"{instruction}"}
            ]

        prompt = pipeline.tokenizer.apply_chat_template(
                messages, 
                tokenize=False, 
                add_generation_prompt=True
        )
        outputs = pipeline(
            prompt,
            eos_token_id=terminators,
            do_sample=True,
            temperature=0.6,
            top_p=0.9
        )
        results.append((row, outputs[0]["generated_text"][len(prompt):]))

In [ ]:
with open('hw4_results/sample_novel_results.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    for res in results:
        writer.writerow(res)

In [ ]:
results = []
with open('hw4_data/sample_science.csv', 'r', newline='') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        instruction = science_prompt.format(question=row)
        messages = [
            {"role": "system", "content": "당신은 유능한 AI 어시스턴트 입니다. 사용자의 질문에 대해 친절하게 답변해주세요."},
            {"role": "user", "content": f"{instruction}"}
            ]

        prompt = pipeline.tokenizer.apply_chat_template(
                messages, 
                tokenize=False, 
                add_generation_prompt=True
        )
        outputs = pipeline(
            prompt,
            eos_token_id=terminators,
            do_sample=True,
            temperature=0.6,
            top_p=0.9
        )
        results.append((row, outputs[0]["generated_text"][len(prompt):]))

In [ ]:
with open('hw4_results/sample_science_results.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    for res in results:
        writer.writerow(res)